In [2]:
import gzip
import pickle

what each data entry means: https://www.hathitrust.org/member-libraries/resources-for-librarians/data-resources/hathifiles/hathifiles-description/ 
Key indices 
- 11 = Title
- 16 = Publication date (if date == 9999 it means it can't be determined from the source)
- 18 = Language (eng = English)
- 25 = author (although not all records have an author)

Right now I'm limiting to books (data[19]== 'bk') as I'm not sure how to handle serials and such (same name but multiple publications) and I want to start with a smaller portion of the dataset
Later I might expand to other kinds of publications (i.e. Serials, electronic resources, maps, music, visual material, etc.)

Originally I was grabbing all file formats but books make up the majority of the data (~6 million with all formats vs )

In [5]:
books = []
bookHash = {}
i = 0
with gzip.open('hathi_full_20240801.txt.gz', 'r') as f:
    for line in f:
        data = line.decode('utf-8').strip().split('\t')
        if data[18] != 'eng' or data[19].lower() != 'bk':
            i += 1
            continue
        title, pubDate =  data[11], data[16]
        if len(data) == 26:
            auth = data[-1]
        else:
            auth = ''
        bookData = [title, pubDate, auth]
        if auth not in bookHash:
            bookHash[auth] = {title: {pubDate: 1}}
            books.append(bookData) # only add unique books to list (diff author and title and publication date)
        else:
            if title not in bookHash[auth]:
                bookHash[auth][title] = {pubDate: 1}
                books.append(bookData) # same author, diff title
            else:
                if pubDate not in bookHash[auth][title]: # same author, same title, diff publication date
                    bookHash[auth][title][pubDate] = 1
                    books.append(bookData)
                else: # assume same book
                    bookHash[auth][title][pubDate] += 1
        i += 1
        if i % 1000000 == 0:
            print(f'{i} records processed')
print(len(books))

1000000 records processed
4000000 records processed
5000000 records processed
8000000 records processed
9000000 records processed


In [4]:
booksWithCount = {}
i = 0
for bookInfo in books:
    title, pubDate, auth = bookInfo
    count = bookHash[auth][title][pubDate]
    booksWithCount[i] = {'title':title, 'date':pubDate, 'author':auth,'copies':count}
    i += 1

In [40]:
with open('englishHathi.pkl', 'wb') as f:
    pickle.dump(booksWithCount, f)